# Exemplo de amostragem

## Bibliotecas

In [1]:
import numpy as np
import pandas as pd
import random as rnd
from sklearn.model_selection import train_test_split  # dividir em treino e teste

## Carregar os dados

In [6]:
dados = pd.read_csv('dados_renda_municipios.csv')

In [7]:
dados.head()

,UF,Municipio,RDPC
0,Rondônia,ALTA FLORESTA D'OESTE,476.99
1,Rondônia,ARIQUEMES,689.95
2,Rondônia,CABIXI,457.17
3,Rondônia,CACOAL,738.06
4,Rondônia,CEREJEIRAS,577.18


## Estudo inicial

In [8]:
estado = 'São Paulo'

In [11]:
dados_municipio = dados[dados['UF'] == estado].reset_index(drop=True)

- Classificação de renda

In [15]:
dados_municipio['classe_renda'] = pd.qcut(dados_municipio['RDPC'],
                                          q = 4,
                                          labels = ['D', 'C', 'B', 'A'])

In [16]:
dados_municipio.head()

,UF,Municipio,RDPC,classe_renda
0,São Paulo,ADAMANTINA,975.43,A
1,São Paulo,ADOLFO,661.65,C
2,São Paulo,AGUAÍ,636.07,C
3,São Paulo,ÁGUAS DA PRATA,853.39,A
4,São Paulo,ÁGUAS DE LINDÓIA,730.13,B


- Medidas gerais da população

In [17]:
dados_piloto = dados_municipio.agg(media_renda = pd.NamedAgg('RDPC', 'mean'),
                                   dp_renda = pd.NamedAgg('RDPC', 'std'),
                                   n_municipios = pd.NamedAgg('RDPC', 'count'))

In [27]:
dados_piloto

,RDPC
media_renda,713.926155
dp_renda,197.398270
n_municipios,645.000000


In [19]:
dados_piloto_classe = dados_municipio.groupby('classe_renda') \
                                     .agg(media_renda = pd.NamedAgg('RDPC', 'mean'),
                                          dp_renda = pd.NamedAgg('RDPC', 'std'),
                                          n_municipios = pd.NamedAgg('RDPC', 'count'))

In [23]:
dados_piloto_classe['percent'] = dados_piloto_classe['n_municipios'] / sum(dados_piloto_classe['n_municipios'])

In [28]:
dados_piloto_classe

,media_renda,dp_renda,n_municipios,percent
classe_renda,,,,
D,517.531296,58.889964,162,0.251163
C,637.659259,28.839032,162,0.251163
B,736.007313,34.142763,160,0.248062
A,966.337453,204.484141,161,0.249612


## Obter o tamanho da amostra

In [29]:
# formula amostra continua
def formula_amostra_continua(ME, N, Z, S):

  n = (Z**2 * S**2 * N) / ((ME**2 * (N-1)) + (Z**2 * S**2))

  return int(n)

In [31]:
# testar se funciona - se 95 ok...
formula_amostra_continua(ME = 100, N = 15000, Z = 1.96, S = 500)

95

In [33]:
ME = 25 # margem de erro
Z = 1.96 # 95% de confiança
S = 197.398270 # da amostra piloto
N = 645 # total de municipios

In [34]:
n = formula_amostra_continua(ME, N, Z, S)
n

174

## Amostragem

- Amostra aleatória simples

In [37]:
dados_amostra_aleatoria = dados_municipio.sample(n=n, # tamanho da amostra
                                                 random_state=1245 # se quiser repetir os mesmos municipios
                                                 )

In [38]:
dados_amostra_aleatoria.shape

(174, 4)

In [39]:
dados_amostra_aleatoria.agg(media_renda = pd.NamedAgg('RDPC', 'mean'),
                            dp_renda = pd.NamedAgg('RDPC', 'std'),
                            n_municipios = pd.NamedAgg('RDPC', 'count'))

,RDPC
media_renda,714.228851
dp_renda,197.027325
n_municipios,174.000000


In [40]:
dados_piloto

,RDPC
media_renda,713.926155
dp_renda,197.398270
n_municipios,645.000000


In [42]:
percentual_amostra = dados_amostra_aleatoria.groupby('classe_renda') \
                       .agg(media_renda = pd.NamedAgg('RDPC', 'mean'),
                            dp_renda = pd.NamedAgg('RDPC', 'std'),
                            n_municipios = pd.NamedAgg('RDPC', 'count'))

In [43]:
percentual_amostra['percent'] = percentual_amostra['n_municipios'] / sum(percentual_amostra['n_municipios'])

In [44]:
percentual_amostra

,media_renda,dp_renda,n_municipios,percent
classe_renda,,,,
D,501.869744,66.576554,39,0.224138
C,639.571136,28.383709,44,0.252874
B,730.668800,33.343062,50,0.287356
A,976.300732,201.544108,41,0.235632


In [45]:
dados_piloto_classe

,media_renda,dp_renda,n_municipios,percent
classe_renda,,,,
D,517.531296,58.889964,162,0.251163
C,637.659259,28.839032,162,0.251163
B,736.007313,34.142763,160,0.248062
A,966.337453,204.484141,161,0.249612


- Amostra estratificada

In [46]:
dados_amostra_estrat = train_test_split(dados_municipio, # os dados totais
                                        test_size = n, # tamanho da amostra
                                        random_state = 1245, # se quiser repetir os valores
                                        stratify = dados_municipio['classe_renda'] # amostra estratificada
                                        )[1]

In [47]:
dados_amostra_estrat.shape

(174, 4)

In [48]:
percentual_amostra_estrat = dados_amostra_estrat.groupby('classe_renda') \
                       .agg(media_renda = pd.NamedAgg('RDPC', 'mean'),
                            dp_renda = pd.NamedAgg('RDPC', 'std'),
                            n_municipios = pd.NamedAgg('RDPC', 'count'))

In [49]:
percentual_amostra_estrat['percent'] = percentual_amostra_estrat['n_municipios'] / sum(percentual_amostra_estrat['n_municipios'])

In [50]:
percentual_amostra_estrat

,media_renda,dp_renda,n_municipios,percent
classe_renda,,,,
D,530.192500,42.159454,44,0.252874
C,635.421591,28.204496,44,0.252874
B,733.693953,34.054255,43,0.247126
A,982.552558,232.962593,43,0.247126


In [51]:
dados_piloto_classe

,media_renda,dp_renda,n_municipios,percent
classe_renda,,,,
D,517.531296,58.889964,162,0.251163
C,637.659259,28.839032,162,0.251163
B,736.007313,34.142763,160,0.248062
A,966.337453,204.484141,161,0.249612
